In [1]:
from materiathena.mp.agent import MPLLM


mpllm = MPLLM()

print("MPLLM is ready to serve you!")


# user_input = input("Enter materials-related question: ")
user_input = "What is the bandgap of stable BaTiO3?"
user_input = "What is the elastic tensor of NaCl?"
user_input = "What is the bandgap of stable MgSe?"
user_input = "Is YbCl3 magnetic or non-magnetic?"
# user_input = "What is the energy above hull of Cu3P4Pb3O16 at 0K?"
# user_input = "Is Cu stable in FCC or BCC structure at 0K based on the energy distance above hull?"
# user_input = "Do we have stable Cu-substituted lead phosphate apatite at 0K?"
# user_input = "What is the bandgap of the most stable magnetite?"
# user_input = "Search for the necessary information for FCC Cu and summarize for me."

mpllm.run_material_conversation(user_input=user_input, debug=True)
# mpllm.run_material_conversation(user_input, model="gpt-3.5-turbo-16k-0613", debug=True)

/global/homes/c/cyrusyc/.conda/envs/dev/lib/python3.11/site-packages/mp_api/client/mprester.py:182: UserWarning: mpcontribs-client not installed. Install the package to query MPContribs data, or construct pourbaix diagrams: 'pip install mpcontribs-client'
  warnings.warn(


MPLLM is ready to serve you!
Conversation started.
{
  "id": "chatcmpl-80hzEzhIPCFbdW8J9faDWzfNWgRjS",
  "object": "chat.completion",
  "created": 1695179776,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Calling Materials Project API..."
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 142,
    "completion_tokens": 5,
    "total_tokens": 147
  }
}
{
  "id": "chatcmpl-80hzF8nHF0DqTqay9F457ac9yoFD0",
  "object": "chat.completion",
  "created": 1695179777,
  "model": "gpt-3.5-turbo-16k-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "search_materials_magnetism__get",
          "arguments": "{\n  \"formula\": \"YbCl3\"\n}"
        }
      },
      "finish_reason": "function_call"
    }
  ],
  "usage": {
    "prompt_tokens": 16190,
    "completion_t

Retrieving MaterialsDoc documents:   0%|          | 0/3 [00:00<?, ?it/s]

Retrieving MagnetismDoc documents:   0%|          | 0/3 [00:00<?, ?it/s]

{
  "id": "chatcmpl-80hzLRFemHPlmfL2lCdx3BdAdcHog",
  "object": "chat.completion",
  "created": 1695179783,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "YbCl3 can exist in both magnetic and non-magnetic forms. There are two magnetic configurations of YbCl3, with the material IDs mp-1025400 and mp-1187659, which exhibit ferromagnetic ordering. These configurations have a total magnetization of approximately 2.0 and 2.0, respectively. On the other hand, there is a non-magnetic configuration of YbCl3 with the material ID mp-1025388. This non-magnetic configuration has a total magnetization of approximately 0.0."
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 581,
    "completion_tokens": 114,
    "total_tokens": 695
  }
}
YbCl3 can exist in both magnetic and non-magnetic forms. There are two magnetic configurations of YbCl3, with the material IDs mp-1025400 a

Retrieving SummaryDoc documents:   0%|          | 0/2 [00:00<?, ?it/s]

{
  "id": "chatcmpl-80hzrtw1Rz3SzMQCPaRkXtzsXdZ6l",
  "object": "chat.completion",
  "created": 1695179815,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "The magnetic configurations of YbCl3, with the material IDs mp-1025400 and mp-1187659, exhibit ferromagnetic ordering. These configurations have a total magnetization of approximately 2.0 and 2.0, respectively. Ferromagnetic materials have a spontaneous magnetization even in the absence of an external magnetic field. YbCl3 in these magnetic configurations shows magnetic properties that can be useful in various applications such as magnetic storage, sensors, and spintronics."
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 252,
    "completion_tokens": 101,
    "total_tokens": 353
  }
}
The magnetic configurations of YbCl3, with the material IDs mp-1025400 and mp-1187659, exhibit ferromagnetic ordering. Thes

InvalidRequestError: The model `gpt-4-32k` does not exist or you do not have access to it. Learn more: https://help.openai.com/en/articles/7102672-how-can-i-access-gpt-4.

# Hide

In [ ]:
from openapi_schema_pydantic import PathItem, Operation, Parameter

# class MPAPISpec:
#     def __init__(
#             self,
#             force_download: bool = False,
#             ):

#         if force_download:
#             spec = OpenAPISpec.from_url("https://api.materialsproject.org/openapi.json")
#             spec.

# TODO: construct MPAPISpec class from OpenAPISpec


def get_endpoints(spec: OpenAPISpec):
    endpoints = [
        (route, method, operation)
        for route, paths in spec.paths.items()
        for method, operation in paths
        if method in ["get", "post"]
    ]
    return endpoints


def get_functions(spec: OpenAPISpec):
    endpoints = get_endpoints(spec)
    functions = []
    for route, method, operation in endpoints:
        if operation is None:
            continue

        name = (
            operation.operationId
            if len(operation.operationId) <= 64
            else operation.operationId[:64]
        )

        description = operation.description or operation.summary

        properties = {
            property.name: {
                "type": property.param_schema.type,
                "description": property.description
                if property.description is not None
                else "",
                **(
                    {"enum": property.param_schema.enum}
                    if property.param_schema.enum is not None
                    else {}
                ),
            }
            for property in operation.parameters
            if property.param_schema.type is not None
        }

        required = [
            property.name for property in operation.parameters if property.required
        ]
        functions.append(
            {
                "name": name,
                "description": description,
                "parameters": {
                    "type": "object",
                    "properties": properties,
                },
                **({"required": required} if required else {}),
            }
        )

    return functions


functions = get_functions(spec)

print(len(functions))

In [ ]:
# import json

# with open("passed_functions.json", "w") as f:
#     json.dump(functions, f, indent=4)

# functions

import json

with open("select_functions.json", "r") as f:
    functions = json.load(f)

len(functions)

In [ ]:
# import json
# import os

# import openai
# from dotenv import load_dotenv
# from mp_api.client import MPRester

# from langchain.tools import APIOperation, OpenAPISpec

# load_dotenv()
# OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", None)
# MP_API_KEY = os.getenv("MP_API_KEY", None)
# openai.api_key = OPENAI_API_KEY

# spec = OpenAPISpec.from_url("https://api.materialsproject.org/openapi.json")
# operation = APIOperation.from_openapi_spec(
#     spec, "/materials/summary/{material_id}/", "get"
# )

# # from langchain.chat_models import ChatOpenAI, ChatAnthropic
# # from langchain.schema import HumanMessage, AIMessage, ChatMessage
# # from langchain.tools import format_tool_to_openai_function

# # llm = ChatAnthropic(anthropic_api_key=)
# # llm = ChatOpenAI(model='gpt-4-0613', temperature=0.5, openai_api_key=OPENAI_API_KEY, client=)


# class LLMaterialsAgent:
#     def __init__(self, mp_api_key=MP_API_KEY, openai_api_key=OPENAI_API_KEY):
#         # Initialize the Materials Project API
#         self.mpr = MPRester(mp_api_key)
#         # Initialize the OpenAI API
#         openai.api_key = openai_api_key

#     def get_materials_data(self, query_params):
#         # Retrieve data from the Materials Project using the MPRester class
#         data = self.mpr.summary.search(**query_params)
#         return data

#     def run_conversation(self, user_input):
#         # Step 1: send the conversation and available functions to GPT
#         messages = [{"role": "user", "content": user_input}]
#         functions = [
#             {
#                 "name": "get_materials_data",
#                 "description": "Get materials data from the Materials Project",
#                 "parameters": {
#                     "type": "object",
#                     "properties": {
#                         "elements": {
#                             "type": "array",
#                             "items": {"type": "string"},
#                             "description": "Elements to query, e.g., ['Si', 'O']",
#                         },
#                         "band_gap": {
#                             "type": "array",
#                             "items": {"type": "number"},
#                             "description": "Range of band gap values, e.g., [0.5, 1.0]",
#                         },
#                         "fields": {
#                             "type": "array",
#                             "items": {"type": "string"},
#                             "description": f"Fields to return, including {self.mpr.summary.available_fields}",
#                         }
#                         # "limit": {
#                         #     "type": "integer",
#                         #     "description": "Number of materials to retrieve, e.g., 5",
#                         # },
#                     },
#                     "required": ["elements", "band_gap"],
#                 },
#             }
#         ]
#         response = openai.ChatCompletion.create(
#             model="gpt-3.5-turbo-0613",
#             messages=messages,
#             functions=functions,
#             function_call="auto",  # auto is default, but we'll be explicit
#         )
#         response_message = response["choices"][0]["message"]

#         # Step 2: check if GPT wanted to call a function
#         if response_message.get("function_call"):
#             # Step 3: call the function
#             # Note: the JSON response may not always be valid; be sure to handle errors
#             available_functions = {
#                 "get_materials_data": self.get_materials_data,
#             }  # only one function in this example, but you can have multiple
#             function_name = response_message["function_call"]["name"]
#             function_to_call = available_functions[function_name]
#             function_args = json.loads(response_message["function_call"]["arguments"])
#             function_response = function_to_call(query_params=function_args)

#             # breakpoint()

#             # Step 4: send the info on the function call and function response to GPT
#             messages.append(
#                 response_message
#             )  # extend conversation with assistant's reply
#             # breakpoint()
#             messages.append(
#                 {
#                     "role": "function",
#                     "name": function_name,
#                     "content": json.dumps(function_response[0]),
#                     # "content": function_response,
#                 }
#             )  # extend conversation with function response
#             # breakpoint()
#             second_response = openai.ChatCompletion.create(
#                 model="gpt-3.5-turbo-0613",
#                 messages=messages,
#             )  # get a new response from GPT where it can see the function response
#             # breakpoint()
#             return second_response

In [ ]:
# user_input = input("Please enter your natural language query: ")

# user_input = "Is UCl3 a stable material according to the thermodynamic hull on Materials Project?"

user_input = "What is the bandgap of diamond based on Materials Project's data?"

messages = [
    {
        "role": "system",
        "content": (
            "You are a data-vigilent agent that responds user requrests based on data hosted on Materials Project."
            "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous. Only use the functions you have been provided with."
            "Remember to provide `fields` or `_fields` to retrive as small request as possible."
        ),
    },
    {"role": "user", "content": user_input},
]


response = openai.ChatCompletion.create(
    # model="gpt-3.5-turbo-0613",
    model="gpt-3.5-turbo-16k-0613",
    # model="gpt-4-0613",
    messages=messages,
    functions=functions,
    function_call="auto",  # auto is default, but we'll be explicit
)
response_message = response["choices"][0]["message"]

In [ ]:
response_message

In [ ]:
# from tqdm.auto import tqdm
# passed_functions = []
# for n in tqdm(range(len(functions))):
#     try:
#         response = openai.ChatCompletion.create(
#             model="gpt-3.5-turbo-0613",
#             messages=messages,
#             functions=[functions[n]],
#             function_call="auto",  # auto is default, but we'll be explicit
#         )
#         response_message = response["choices"][0]["message"]
#     except Exception as e:
#         print(functions[n]["name"])
#         continue
#     passed_functions.append(functions[n])

In [ ]:
# list(map(lambda x: x['name'], functions)).index('search_materials_synthesis__get')

# functions.pop(47)

In [ ]:
# import json

# with open("passed_functions.json", "w") as f:
#     json.dump(functions, f, indent=4)

# functions

In [ ]:
list(map(lambda x: x["name"], functions)).index("search_materials_summary__get")

from langchain.chat_models import ChatOpenAI, ChatAnthropic




model = ChatOpenAI(model_name="gpt-4", temperature=0.0)

In [ ]:
from mp_api.client.core import BaseRester
from mp_api.client.routes import *


class MPLLM:
    def __init__(self, mp_api_key=MP_API_KEY, openai_api_key=OPENAI_API_KEY) -> None:
        openai.api_key = openai_api_key

        self.base = BaseRester(
            api_key=mp_api_key, monty_decode=True, use_document_model=False
        )

        self.materials = MaterialsRester(
            api_key=mp_api_key, monty_decode=False, use_document_model=False
        )

        self.thermo = ThermoRester(
            api_key=mp_api_key, monty_decode=False, use_document_model=False
        )

        self.tasks = TaskRester(
            api_key=mp_api_key,
        )

        self.summary = SummaryRester(
            api_key=mp_api_key, monty_decode=False, use_document_model=False
        )

    def search_base(self, query_params):
        query_params["fields"] = query_params.get("fields", "").split(",")
        _fields = query_params.pop("_fields", None)
        if _fields:
            query_params["fields"] += _fields.split(",")
        return self.base._search(all_fields=False, **query_params)

    def search_materials_core(self, query_params: dict):
        query_params["fields"] = query_params.get("fields", "").split(",")
        _fields = query_params.pop("_fields", None)
        if _fields:
            query_params["fields"] += _fields.split(",")
        return self.materials._search(all_fields=False, **query_params)

    def search_materials_summary(self, query_params: dict):
        query_params["fields"] = query_params.get("fields", "").split(",")
        _fields = query_params.pop("_fields", None)
        if _fields:
            query_params["fields"] += _fields.split(",")
        return self.summary._search(all_fields=False, **query_params)

    def search_materials_thermo(self, query_params):
        query_params["fields"] = query_params.get("fields", "").split(",")
        _fields = query_params.pop("_fields", None)
        if _fields:
            query_params["fields"] += _fields.split(",")
        return self.thermo._search(all_fields=False, **query_params)


mpllm = MPLLM()

In [ ]:
available_functions = {
    "search_base": mpllm.search_base,
    "search_materials_thermo__get": mpllm.search_materials_thermo,
    "search_materials_core__get": mpllm.search_materials_core,
    "search_materials_summary_stats__get": mpllm.search_materials_core,
    "search_materials_summary__get": mpllm.search_materials_summary,
}  # only one function in this example, but you can have multiple
function_name = response_message["function_call"]["name"]
function_to_call = available_functions[function_name]
function_args = json.loads(response_message["function_call"]["arguments"])
function_response = function_to_call(query_params=function_args)

In [ ]:
print(function_to_call)
print(function_args)

response_message["function_call"]["arguments"]

In [ ]:
from monty.json import MontyEncoder, MontyDecoder

# json.dumps(function_response[0], cls=MontyEncoder)
# json.dumps(function_response[0].dict(), indent=4, cls=MontyDecoder)
# function_response[0]
# MontyEncoder().encode(function_response[0])
# MontyDecoder().decode(function_response[0].__str__())

In [ ]:
function_response[0]

In [ ]:
content = json.dumps(function_response)

messages.append(
    {
        "role": "function",
        "name": function_name,
        "content": content,
        # "content": function_response,
    }
)

In [ ]:
second_response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
)  # get a new response from GPT where it can see the function response

In [ ]:
second_response

In [ ]:
from pymatgen.core import Structure, Element
import datetime


def serialize_nested_object(obj):
    if isinstance(obj, dict):
        serialized_dict = {}
        for key, value in obj.items():
            serialized_dict[key] = serialize_nested_object(value)
        return serialized_dict
    elif isinstance(obj, list):
        serialized_list = []
        for item in obj:
            serialized_list.append(serialize_nested_object(item))
        return serialized_list
    elif isinstance(obj, (datetime.datetime, Element)):
        return str(obj)
    elif isinstance(obj, (str, int, float)):
        return obj
    else:
        # Handle other data types or custom objects as needed
        return str(obj)


serialize_nested_object(function_response[0].dict())

In [ ]:
from monty.json import MontyEncoder

encoder = MontyEncoder()
a = encoder.encode(function_response[0])

In [ ]:
from monty.json import MontyDecoder, MontyEncoder

decoder = MontyDecoder()
decoder.decode(function_response[0].dict())
# encoder = MontyEncoder()
# encoder.encode()
# json.dumps(function_response[0])